In [84]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [124]:
## lets fit a NNet model for continuous variable, 


data1 = pd.read_csv("red-wine.csv") ## read data
data2 = data1.loc[[356]]
data = data1.append(data2) # making data 1600 columns for ease of split


train, test = train_test_split(data, test_size= 2/16) ## splitting data into train and test

X = train.drop("quality",1).values                       ##########
Y = np.reshape(train["quality"].values,(1400,1))         #####
X_test = test.drop("quality",1).values                   #####       BUNCH OF DATA STRUCTURE MANIPULATION
Y_test = np.reshape(test["quality"].values,(200,1))      ##########

batch_size = 200
size = batch_size

cursor = 0

def feed(batch_size):                                       ################ Function to feed batches of data
    assert size%batch_size == 0 ##to make sure perfect allocation of data
    global cursor
    x_train = X[cursor:cursor+batch_size]
    y_train = Y[cursor:cursor+batch_size]
    if cursor == size  :
        cursor = 0
    else :
        cursor += batch_size
    return x_train, y_train

In [125]:
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, 11))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,1 ))
    
    tf_test_dataset = tf.placeholder(tf.float32,
                                    shape=(200, 11))
    tf_test_labels = tf.placeholder(tf.float32, shape=(batch_size,1 ))
    
    
    weights1 = tf.Variable(
    tf.truncated_normal([11, 32 ]))
    biases1 = tf.Variable(tf.zeros([32]))
    weights2 = tf.Variable(
    tf.truncated_normal([32,1 ]))
    biases2 = tf.Variable(tf.zeros([1]))
    # Training computation.
    logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
    relu1 =  tf.nn.relu(logits1)
    logits2 = tf.matmul(relu1, weights2) + biases2

    loss = tf.losses.mean_squared_error(labels=tf_train_labels, predictions=logits2,weights=1.0) 
    optimizer = tf.train.AdagradOptimizer(0.5).minimize(loss)
    
    tlogits1 = tf.matmul(tf_test_dataset, weights1) + biases1
    trelu1 =  tf.nn.relu(logits1)
    tlogits2 = tf.matmul(relu1, weights2) + biases2

    tloss = tf.losses.mean_squared_error(labels=tf_test_labels, predictions=tlogits2,weights=1.0) 
  

In [128]:
num_steps = 300000
with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        
        batch_data, batch_labels = feed(batch_size)
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_test_dataset : X_test, tf_test_labels : Y_test}
        _, l , j = session.run([optimizer, loss,tloss], feed_dict=feed_dict)
        if (step % 20000 == 0):
            print("Minibatch loss(MSE) at step %d: %f" % (step, l))
            print("test loss(MSE) at step %d: %f" % (step, j))
        

            
        
        

Initialized
Minibatch loss(MSE) at step 0: 15281.459961
test loss(MSE) at step 0: 15276.865234
Minibatch loss(MSE) at step 20000: 0.439228
test loss(MSE) at step 20000: 1.192688
Minibatch loss(MSE) at step 40000: 0.358429
test loss(MSE) at step 40000: 1.103507
Minibatch loss(MSE) at step 60000: 0.337588
test loss(MSE) at step 60000: 1.073454
Minibatch loss(MSE) at step 80000: 0.326294
test loss(MSE) at step 80000: 1.055665
Minibatch loss(MSE) at step 100000: 0.319310
test loss(MSE) at step 100000: 1.044339
Minibatch loss(MSE) at step 120000: 0.314602
test loss(MSE) at step 120000: 1.036888
Minibatch loss(MSE) at step 140000: 0.311225
test loss(MSE) at step 140000: 1.031749
Minibatch loss(MSE) at step 160000: 0.308663
test loss(MSE) at step 160000: 1.028153
Minibatch loss(MSE) at step 180000: 0.306523
test loss(MSE) at step 180000: 1.025730
Minibatch loss(MSE) at step 200000: 0.304812
test loss(MSE) at step 200000: 1.023852
Minibatch loss(MSE) at step 220000: 0.303359
test loss(MSE) at 

200